In [1]:
import thrember
thrember.download_dataset("C:\\Users\\Thinkbook 14\\AIMaP\\data\\ember_data")

Win32_train.zip:   0%|          | 0.00/12.3G [00:00<?, ?B/s]

c:\Users\Public\conda\envs\aimap\lib\site-packages\huggingface_hub\file_download.py:121: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Thinkbook 14\.cache\huggingface\hub\datasets--joyce8--EMBER2024. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Unzipping...


Win64_train.zip:   0%|          | 0.00/5.85G [00:00<?, ?B/s]

Unzipping...


...

ELF_test.zip:   0%|          | 0.00/16.8M [00:00<?, ?B/s]

Unzipping...


PDF_test.zip:   0%|          | 0.00/28.8M [00:00<?, ?B/s]

Unzipping...


challenge.zip:   0%|          | 0.00/32.1M [00:00<?, ?B/s]

In [2]:
import os
import thrember
import numpy as np
import pandas as pd
import polars as pl
import altair as alt
import lightgbm as lgb
import matplotlib.pylab as plt
from sklearn.metrics import roc_auc_score, roc_curve
alt.renderers.enable('default')

RendererRegistry.enable('default')

In [3]:
data_dir = "C:\\Users\\Thinkbook 14\\AIMaP\\data\\ember_data"

In [ ]:
train_df, test_df, challenge_df = thrember.read_metadata(data_dir)

In [ ]:
# Add a 'week' column to the dataframe
plotdf = pl.concat([train_df, test_df])
start_date = pd.Timestamp("2023-09-24")
plotdf = plotdf.with_columns(
    pl.from_epoch("first_submission_date", time_unit="s").alias("first_submission_dt")
)
plotdf = plotdf.with_columns(
    (
        (pl.col("first_submission_dt") - pl.lit(start_date)).dt.total_days() // 7
    ).cast(pl.Int64).alias("week")
)

print(plotdf.shape)

# Plot file types across weeks
gbdf = plotdf.group_by(["file_type", "week"]).agg(pl.len().alias("count"))
alt.Chart(gbdf).mark_bar().encode(
    alt.X('week:O', axis=alt.Axis(title='Week First Seen')),
    alt.Y('count:Q', axis=alt.Axis(title='File Type')),
        alt.Color('file_type:N', scale=alt.Scale(range=["#4c78a8", "#54a24b", "#f58518",  "#88d27a",  "#9ecae9", "#ffbf79"]),
              legend=alt.Legend(values=["Win32", "Win64", "Dot_Net", "APK", "ELF", "PDF"]))
)

In [ ]:
# Get number of occurrences of each family
family_counts = plotdf.select(
    pl.col("family").value_counts().alias("family_counts")
).unnest("family_counts")
family_counts = (
    plotdf.filter(pl.col("family").is_not_null())
          .select(pl.col("family").value_counts())
          .unnest("family")
          .sort("count", descending=True)
)
family_counts.head(10)

In [ ]:
# Get number of occurrences of each behavior tag
plotdf_explode = plotdf.filter(pl.col("behavior").list.len() > 0).explode("behavior")
behavior_counts = (plotdf_explode.group_by("behavior").agg(pl.len().alias("count")).sort("count", descending=True))
behavior_counts.head(10)